# Vessel Route Prediction using Bidirectional GRU Neural Network

This Jupyter notebook focuses on the preprocessing of vessel position and trajectory data, obtained through an antenna located on the roof of the University of Piraeus Central Building. Our primary objective is to train a neural network, utilizing masking layers and bidirectional GRU, to analyze and predict vessel routes accurately.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings

warnings.filterwarnings("ignore")

In [ ]:
trips =pd.read_csv('../Dataset_after/trips.csv')
trajectories = pd.read_csv('../Dataset_after/trajectories.csv')
positions = pd.read_csv('cleaned_data.csv')

## Pre-Processing

In [ ]:
trajectories.dtypes

It seems that we'll need to convert the start_time & end_time to datetime.

In [ ]:
trajectories['start_time'] = pd.to_datetime(trajectories['start_time'])
trajectories['end_time'] = pd.to_datetime(trajectories['end_time'])

trajectories.dtypes

In [ ]:
trajectories['status'].value_counts()

We don't care about trajectories of status 1 & 5 because we want to have moving vessels only.

In [ ]:
trajectories = trajectories[(trajectories['status'] != 1 )& (trajectories['status'] != 5 )]
trajectories['status'].value_counts()

Clean the columns we won't need.

In [ ]:
positions.drop(columns=['geom','geometry','location_geometry','Unnamed: 0'], inplace=True)
positions.columns

We will only keep the vessels that appeared in **Piraeus Port, which is the port with code 23** and the **type of these vessels are 60-69, which is the passenger vessels.**

In [ ]:
piraeus_passengers = list(positions[(positions['port_code'] == 23) & (positions['type'] >= 60) & (positions['type'] <= 69)]['vessel_id'].unique())

Keep the vessel position with velocity in knots within the range (1,50), because we want them to be sure that they moving and also it is very rare to have vessels with velocity more than 50 knots.

In [ ]:
positions= positions.loc[positions['velocity in knots'].between(1,50, inclusive='neither')].copy()
len(positions)

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot( positions['velocity in knots'])
plt.show()

We will drop as outliers, the positions where the vessels had over 30 knots speed, because we can see that most of the vessels have speed under 30 knots:

pos_moving = positions.loc[positions['velocity in knots'].between(1,30, inclusive='neither')].copy()
pos_moving

We will create a GeoDataframe in **crs = 4326** and then transform it to **crs = 2100** because we want to have it in Greek CRS where the coordinates are calclulated in meters. We will use the meter calculated coordinates to then create a loss function in our neural network that calclulates the loss as the **Euclidean Distance** between the prediction and the real position.

In [ ]:
gdf = gpd.GeoDataFrame(pos_moving, geometry=gpd.points_from_xy(pos_moving['lon'], pos_moving['lat']), crs=4326)
gdf

In [ ]:
gdf = gdf.to_crs('EPSG:2100')
gdf

In [ ]:
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y
gdf

Drop the vessel positions of status 1 & 5, beause we only want vessels that are moving for our location prediction problem (check the statuse here: https://help.marinetraffic.com/hc/en-us/articles/203990998-What-is-the-significance-of-the-AIS-Navigational-Status-Values-)

In [ ]:
gdf = gdf[(gdf['Status'] != 1 ) & (gdf['Status'] != 5 )]
gdf.drop(columns=['lon','lat'], inplace = True)
gdf

In [ ]:
gdf.port_code.value_counts()

Every vessel position is out of port, in the open sea.

In [ ]:
gdf.dtypes

Make the timestamp t datetime:

In [ ]:
gdf['t'] = pd.to_datetime(gdf['t'])
gdf.dtypes

Let's keep only the trajectories that have a start and an end time.

In [ ]:
filtered_trajectories = trajectories[(~trajectories['start_time'].isna()) & (~trajectories['end_time'].isna())]
filtered_trajectories.drop(columns=['Unnamed: 0'], inplace=True)
filtered_trajectories

Now keep only the **positions of the vessels that appeared in the Piraeus Port.**

In [ ]:
gdf_passenger = gdf[gdf['vessel_id'].isin(piraeus_passengers)]
gdf_passenger

Drop the rows with null course, because this feature is important for the location prediction.

gdf_passenger['course'].isna().sum()

In [ ]:
gdf_passenger.dropna(subset=['course'],inplace= True)

In [ ]:
gdf_passenger['course'].isna().sum()

In [ ]:
len(piraeus_passengers)

We have 35 unique vessels, lets plot a sample of these positions.

In [ ]:
gdf_passenger[['latitude','geometry']].sample(30000).explore()

We can see that the trajectories seem to be moving from Piraeus to 3 main routes and back. Let's now keep only the trajectories of these specific 35 vessels.

trajectories_pass = filtered_trajectories[filtered_trajectories['vessel_id'].isin(piraeus_passengers)]
trajectories_pass

In [ ]:
trajectories_pass = trajectories_pass.sample(frac=1) # shuffle the dataframe
trajectories_pass.reset_index(drop= True, inplace=True)
trajectories_pass

We will create a trajectory generator, which we'll use to enter every position sub gdf of each trajectory in a column named segments.

In [ ]:
def trajectory_generator(trajectories,gdf):
        for _, traj in trajectories.iterrows():
                start = traj['start_time']
                end = traj['end_time']
                vessel = traj['vessel_id']
                subgdf = gdf[gdf['vessel_id'] == vessel]
                seg = subgdf[(subgdf['t'] >= start) & (subgdf['t'] <= end)]
                if len(seg) ==0:
                        yield gpd.GeoDataFrame()
                else:
                        yield seg


segments = []
for seg in trajectory_generator(trajectories_pass,gdf_passenger):
    segments.append(seg)

**Now for every trajectory we have a corresponding segment column, that shows every vessels' position for that trajectory.**

In [ ]:
trajectories_pass['segments'] = segments
trajectories_pass

Let's keep the trajectories that haven't got empty positions.

In [ ]:
traj_pos = trajectories_pass[trajectories_pass['segments'].apply(len) > 0].copy()

traj_pos.reset_index(inplace=True, drop = True)

traj_pos

Now we will check the duration of each trajectory in order to see if there are trajectories that lasted way to long.

In [ ]:
traj_pos['duration'] = (traj_pos['end_time'] - traj_pos['start_time']).dt.total_seconds() / 60
traj_pos

In [ ]:
plt.boxplot(traj_pos['duration']);

In [ ]:
plt.boxplot(traj_pos['duration'],showfliers = False);

We will only the trajectories with duration over the 25% quantile of the dataset durations and with at least 30 positions in each trajectory.

In [ ]:
traj_pos= traj_pos[traj_pos['duration'] >= traj_pos['duration'].quantile(0.25)]
traj_pos = traj_pos[traj_pos['segments'].apply(len) > 20]
traj_pos.reset_index(drop = True,inplace=True)
traj_pos

Now we will apply a timestamp in seconds for every position in every trajectory in order to then use it to calclulate velocity, interpolated dt etc.

In [ ]:
from datetime import datetime

def create_seconds(segment):
    epoch_time = datetime(1970, 1, 1)
    segment['time_sec'] = (segment['t'] - epoch_time).dt.total_seconds()
    return segment

In [ ]:
for segment in traj_pos['segments']:
    segment = create_seconds(segment)

Example of a trajectory segment:

In [ ]:
traj_pos.loc[5,'segments'].head()

Reset the indexes:

In [ ]:
for segment in traj_pos['segments']:
    segment.reset_index(drop=True, inplace=True)

**We can see that in every trajectory there is a Δt that is very big. This is due to the loss of the signal near Aigina Island. So we want to split every existing trajectory into 2 sub trajectories (1 heading south and 1 north).**

In [ ]:
maxes = []
for segment in traj_pos['segments']:
    maxes.append(segment['Δt'].max())

np.max(maxes)

In [ ]:
max_dt = []
for i in range(len(traj_pos)):
    max_dt.append(max(traj_pos.loc[i,'segments']['Δt'].value_counts().keys()))

plt.boxplot(max_dt, showfliers=True);

In [ ]:
plt.boxplot(max_dt, showfliers=False);

In [ ]:
np.mean(max_dt)

Recalculate the time difference between the timestamps for every segment and set the first dt = 0 :

In [ ]:
for segment in traj_pos['segments']:
    segment['dt'] = segment['time_sec'].diff()

In [ ]:
for segment in traj_pos['segments']:
    segment.loc[0,'dt'] = 0

Let's check 1 random trajectory - segment:

In [ ]:
import random
idx = random.randint(0,len(traj_pos)-1)
random_segment = traj_pos.loc[idx,'segments']
gdf_temp = gpd.GeoDataFrame(random_segment, geometry=gpd.points_from_xy(random_segment.loc[:,'longitude'], random_segment.loc[:,'latitude']), crs=2100)
gdf_temp[['time_sec','dt','id_x','geometry']].explore()

As the map indicates, when the vessel was leaving from Piraeus heading south, we lost its signals. The signals returned when it was heading back to Piraeus 17363 seconds after the previous signal! **Here comes the need to split every trajectory into 2 subtrajectories (1 moving away from Piraeus and 1 moving to Piraeus) because this happens to all of our trajectories.**

In [ ]:
new_segs = []
threshold = 300 # 5 minutes without signal -> split the trajectory (don't interpolate in other words)
for seg in traj_pos['segments']:

    idx_sep = list(seg[seg['dt'] > threshold].index)
    
    start = 0
    counter = 0
    for i in range(len(idx_sep)):
        end = idx_sep[i]
        new_segs.append(seg.iloc[start:end])
        start = end
        counter += 1
        if counter == len(idx_sep):
            new_segs.append(seg.iloc[start:])

In [ ]:
new_traj_data = pd.DataFrame(new_segs,columns=['segments'])

In [ ]:
df_traj = new_traj_data[new_traj_data['segments'].apply(len) > 20]
df_traj.reset_index(inplace=True,drop = True)
df_traj


Let's check again a random trajectory-segment:

In [ ]:
idx = random.randint(0,len(df_traj)-1)
random_segment = df_traj.loc[idx,'segments']
gdf_temp = gpd.GeoDataFrame(random_segment, geometry=gpd.points_from_xy(random_segment.loc[:,'longitude'], random_segment.loc[:,'latitude']), crs=2100)
gdf_temp[['time_sec','course','dt','id_x','geometry']].explore()

So we can see that now we have a single real trajectory and not 2 subtrajectories inside 1. Now reset again the indexes.

In [ ]:
for segment in df_traj['segments']:
    segment.reset_index(drop=True, inplace= True)

**We will split the dataset to training and test. The training dataset will be the 80% of the trajectories of the vessel and the test dataset will be the remaining 20% trajectories**

In [ ]:
sep = int(len(df_traj)*0.75)
traj_train = df_traj[:sep]
traj_test = df_traj[sep:]
traj_train

# Prepairing Data for the Model.<br>
We will start by creating the following generators and functions:

- **get_segments**: we will use this generator to iritate through the sub-trajectories and not having them loaded on memory.

- **calculate_distance**: It calclulates the Euclidean Distance between two points in EPGS 2100

- **interpolate** : We will use this function to apply linear interpolation in order to have synchronized timestamps timestamps

- **calculate_dt**: It calculates the time difference between two signals/timestamps.

- **balance_trajectories**: It is the function that returns the trajectory positions balanced, with synchronized timestamps.<br><br>

The features we will are the following:

- **lon** - Longitude

- **lat** - Latitude

- **timestamp** - Timestamp

- **velocity** - Speed of vessels in that timestamp calclulated in m/s

- **distance** - The distance of the vessel from its previous position

- **distance_AIS** - The distance of the vessel from the Antenna (University of Piraeus)<br><br>

Here are the University of Piraues coordinates that we will use for our antenna distance calclulation:

**University of Piraeus in EPGS:4326** (23.647269,37.9438708)

**University of Piraeus in EPGS:2100** (468857.80056930066,4199358.117602245)

In [ ]:
unipi = [468857.80056930066,4199358.117602245]